In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
# prompt: Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: load X_train.npy and Y.npy and train validation and test split

import numpy as np
from sklearn.model_selection import train_test_split

X = np.load("/content/drive/MyDrive/X_array.npy")
Y = np.load("/content/drive/MyDrive/Y.npy")

# Split into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Further split validation set into validation and test sets (optional)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=42)

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (590, 330750)
Y_train shape: (590,)
X_val shape: (74, 330750)
Y_val shape: (74,)
X_test shape: (74, 330750)
Y_test shape: (74,)


In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [ ]:
# prompt: Convert X_train and Y_train to Dataset({
#     features: [ 'audio', 'genre'],
#     num_rows:
# })
from datasets import Dataset, Audio
import soundfile as sf
import io
def convert_to_dataset(X, Y, sampling_rate):
  """Converts X_train and Y_train to a Dataset with 'audio' and 'genre' features."""

  def convert_to_bytes(audio_data, sampling_rate):
    """Converts audio data to bytes."""
    buffer = io.BytesIO()
    sf.write(buffer, audio_data, sampling_rate, format='WAV')
    audio_bytes = buffer.getvalue()
    return audio_bytes

  X_train_bytes = [convert_to_bytes(audio, 22050) for audio in X]
  # X_train_bytes = X
  listY = Y.tolist()
  int_list = [int(x) for x in listY]
  dataset = Dataset.from_dict({"audio": X_train_bytes, "genre": int_list})
  dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
  return dataset


train_dataset = convert_to_dataset(X_train, Y_train, 16000)
# print(train_dataset)

In [ ]:
val_dataset = convert_to_dataset(X_val, Y_val, 16000)
test_dataset = convert_to_dataset(X_test, Y_test, 16000)

In [ ]:
train_dataset[0]

{'audio': {'path': None,
  'array': array([ 0.0003983 , -0.00210748, -0.004734  , ...,  0.03223047,
          0.02570619,  0.02562226]),
  'sampling_rate': 16000},
 'genre': 1}

In [ ]:
max_duration = 15


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
X_train_encoded =  train_dataset.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
X_train_encoded

Map:   0%|          | 0/590 [00:00<?, ? examples/s]

Dataset({
    features: ['genre', 'input_values', 'attention_mask'],
    num_rows: 590
})

In [ ]:
X_val_encoded =  val_dataset.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
X_val_encoded

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Dataset({
    features: ['genre', 'input_values', 'attention_mask'],
    num_rows: 74
})

In [ ]:
X_test_encoded =  test_dataset.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
X_test_encoded

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Dataset({
    features: ['genre', 'input_values', 'attention_mask'],
    num_rows: 74
})

In [ ]:
X_train_encoded = X_train_encoded.rename_column("genre", "label")
X_val_encoded = X_val_encoded.rename_column("genre", "label")
X_test_encoded = X_test_encoded.rename_column("genre", "label")

In [ ]:
label2id = {
    'disco': 0,
    'blues': 1,
    'hiphop': 2,
    # Add the rest of your genres here...
    'classical': 3,
    'metal': 4,
    'jazz': 5,
    'reggae': 6,
    'rock': 7,
    'country': 8,
    'pop': 9
}


In [ ]:
# prompt: generate ud2 label

id2label = {v: k for k, v in label2id.items()}

# Now you can use id2label to get the label from its ID
# For example, if you have the ID 0, you can get the label using id2label[0]
print(id2label)


{0: 'disco', 1: 'blues', 2: 'hiphop', 3: 'classical', 4: 'metal', 5: 'jazz', 6: 'reggae', 7: 'rock', 8: 'country', 9: 'pop'}


In [ ]:
from transformers import AutoModelForAudioClassification

num_labels = 10

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 20

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=X_train_encoded,
    eval_dataset=X_val_encoded,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,2.212100,2.198730,0.351351
2,1.817000,1.758875,0.594595
3,1.556200,1.316238,0.675676
4,1.091900,0.951042,0.824324
5,0.767800,0.773400,0.797297
6,0.468300,0.608351,0.864865
7,0.342000,0.565157,0.783784
8,0.310300,0.523529,0.797297
9,0.139300,0.523300,0.851351
10,0.171100,0.608659,0.851351


TrainOutput(global_step=1480, training_loss=0.4774804583096222, metrics={'train_runtime': 2908.2313, 'train_samples_per_second': 4.057, 'train_steps_per_second': 0.509, 'total_flos': 4.02564302784e+17, 'train_loss': 0.4774804583096222, 'epoch': 20.0})

In [ ]:
results = trainer.evaluate(eval_dataset=X_test_encoded)
print(results)

{'eval_loss': 1.0202158689498901, 'eval_accuracy': 0.7027027027027027, 'eval_runtime': 15.431, 'eval_samples_per_second': 4.796, 'eval_steps_per_second': 0.648, 'epoch': 20.0}


In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForAudioClassification

num_labels = 10

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=X_train_encoded,
    eval_dataset=X_val_encoded,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,2.087600,2.034655,0.418919
2,1.512500,1.461485,0.716216
3,1.295200,1.099962,0.770270
4,1.030000,0.906116,0.810811
5,0.781400,0.777911,0.783784
6,0.496300,0.620384,0.810811
7,0.481400,0.555268,0.851351
8,0.381700,0.513266,0.851351
9,0.286300,0.493636,0.837838
10,0.319700,0.487915,0.837838


TrainOutput(global_step=740, training_loss=0.9082117579273276, metrics={'train_runtime': 1464.183, 'train_samples_per_second': 4.03, 'train_steps_per_second': 0.505, 'total_flos': 2.01282151392e+17, 'train_loss': 0.9082117579273276, 'epoch': 10.0})

In [ ]:
results = trainer.evaluate(eval_dataset=X_test_encoded)
print(results)

{'eval_loss': 0.8802935481071472, 'eval_accuracy': 0.7567567567567568, 'eval_runtime': 15.0406, 'eval_samples_per_second': 4.92, 'eval_steps_per_second': 0.665, 'epoch': 10.0}


In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForAudioClassification

num_labels = 10

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 5

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=X_train_encoded,
    eval_dataset=X_val_encoded,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,2.029700,1.925214,0.459459
2,1.576700,1.455058,0.729730
3,1.427100,1.220472,0.689189
4,1.225000,1.070943,0.756757
5,1.022700,1.007186,0.756757


TrainOutput(global_step=370, training_loss=1.5244679515426223, metrics={'train_runtime': 735.5786, 'train_samples_per_second': 4.01, 'train_steps_per_second': 0.503, 'total_flos': 1.00641075696e+17, 'train_loss': 1.5244679515426223, 'epoch': 5.0})

In [ ]:
results = trainer.evaluate(eval_dataset=X_test_encoded)
print(results)

{'eval_loss': 1.2634409666061401, 'eval_accuracy': 0.581081081081081, 'eval_runtime': 15.1595, 'eval_samples_per_second': 4.881, 'eval_steps_per_second': 0.66, 'epoch': 5.0}


In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForAudioClassification

num_labels = 10

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=X_train_encoded,
    eval_dataset=X_val_encoded,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,2.112900,2.057472,0.405405
2,1.562900,1.600071,0.648649
3,1.432800,1.189869,0.783784
4,1.036200,0.949568,0.824324
5,0.779100,0.783901,0.864865
6,0.527500,0.658500,0.864865
7,0.479200,0.609519,0.864865


Epoch,Training Loss,Validation Loss,Accuracy
1,2.112900,2.057472,0.405405
2,1.562900,1.600071,0.648649
3,1.432800,1.189869,0.783784
4,1.036200,0.949568,0.824324
5,0.779100,0.783901,0.864865
6,0.527500,0.658500,0.864865
7,0.479200,0.609519,0.864865
8,0.394500,0.546731,0.864865
9,0.292800,0.558590,0.851351
10,0.326800,0.550868,0.851351


TrainOutput(global_step=740, training_loss=0.9490655980400137, metrics={'train_runtime': 1461.7034, 'train_samples_per_second': 4.036, 'train_steps_per_second': 0.506, 'total_flos': 2.01282151392e+17, 'train_loss': 0.9490655980400137, 'epoch': 10.0})

TrainOutput(global_step=740, training_loss=0.9490655980400137, metrics={'train_runtime': 1461.7034, 'train_samples_per_second': 4.036, 'train_steps_per_second': 0.506, 'total_flos': 2.01282151392e+17, 'train_loss': 0.9490655980400137, 'epoch': 10.0})

In [ ]:
results = trainer.evaluate(eval_dataset=X_test_encoded)
print(results)

{'eval_loss': 0.9545733332633972, 'eval_accuracy': 0.7162162162162162, 'eval_runtime': 15.1233, 'eval_samples_per_second': 4.893, 'eval_steps_per_second': 0.661, 'epoch': 10.0}
{'eval_loss': 0.9545733332633972, 'eval_accuracy': 0.7162162162162162, 'eval_runtime': 15.1233, 'eval_samples_per_second': 4.893, 'eval_steps_per_second': 0.661, 'epoch': 10.0}


In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForAudioClassification

num_labels = 10

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=X_train_encoded,
    eval_dataset=X_val_encoded,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,2.119100,2.047033,0.418919
2,1.621900,1.479710,0.729730
3,1.395700,1.177602,0.783784
4,1.107900,0.950548,0.797297
5,0.784000,0.801301,0.797297
6,0.565500,0.631679,0.864865
7,0.480900,0.570735,0.864865
8,0.503400,0.536076,0.851351
9,0.370100,0.497771,0.864865
10,0.337500,0.500225,0.837838


TrainOutput(global_step=740, training_loss=0.9651254971285124, metrics={'train_runtime': 1470.3742, 'train_samples_per_second': 4.013, 'train_steps_per_second': 0.503, 'total_flos': 2.01282151392e+17, 'train_loss': 0.9651254971285124, 'epoch': 10.0})

In [ ]:
results = trainer.evaluate(eval_dataset=X_test_encoded)
print(results)

{'eval_loss': 0.9191020131111145, 'eval_accuracy': 0.6891891891891891, 'eval_runtime': 14.9997, 'eval_samples_per_second': 4.933, 'eval_steps_per_second': 0.667, 'epoch': 10.0}
